# Trade use trend strategy

In [1]:
import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import artool
from artool import toy, analyze
from artool.toy.toy_simu import get_pnl_simple

# remove limits on number of rows and columns
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

import matplotlib
matplotlib.use("Agg")  # use to improve performance
import matplotlib.pyplot as plt
import seaborn as sns

## Data


In [2]:
data_dir = Path("/home/yangzhe/data/toy_data_2")
date_start = datetime.datetime(2022, 1, 1)
date_end = datetime.datetime(2022, 9, 1)
symbols = toy.toy_data.get_symbol_list(date_start, date_end, logic="and")
print(f"number of symbols: {len(symbols)}")

df = pd.DataFrame()
for symbol in symbols:
    df_ = pd.read_feather(data_dir / f"{symbol}.feather")
    # remove head/tail 5 rows
    df_ = df_.iloc[5:-5]
    df = pd.concat([df, df_], axis=0)
df = df.reset_index(drop=True)

number of symbols: 129


In [3]:
train_end_date = datetime.datetime(2022, 6, 1)
train_idx = df["funding_timestamp"] < train_end_date.timestamp() * 1e6
test_idx = ~train_idx
df_train = df[train_idx]
df_test = df[test_idx]

x_feature = "funding_rate_expcumsum_5"
y_feature = "funding_rate_future_5"

buy_dim = np.linspace(0, 5e-4, 20)
sell_dim = np.linspace(-1e-4, 1e-4, 20)

## Method 0

Use all symbols available.

In [7]:
method_0_dir = Path("./method_0")
method_0_dir.mkdir(exist_ok=True)

toy

In [8]:
buythres = 1e-4
sellthres = 0
total_pnl = 0
for symbol in symbols:
    x = df_test.loc[df_test["symbol"] == symbol, x_feature].values
    y = df_test.loc[df_test["symbol"] == symbol, y_feature].values
    fr = df_test.loc[df_test["symbol"] == symbol, "funding_rate"].values
    cur_pnl = get_pnl_simple(x, fr, buythres, sellthres)
    total_pnl += cur_pnl
pnl_rate = total_pnl / len(symbols)
print(f"total_pnl: {total_pnl}, n_symbols: {len(symbols)}")
print(f"pnl_rate (y): {pnl_rate * 4 * 100} %")

total_pnl: 0.48965671000000005, n_symbols: 129
pnl_rate (y): 1.5183153798449613 %


Scan train to get optimal thresould

In [9]:
# scan buythres and sellthres
dims = {"buythres": buy_dim, "sellthres": sell_dim}
def obj_train(buythres, sellthres):
    total_pnl = 0
    for symbol in symbols:
        x = df_train.loc[df_train["symbol"] == symbol, x_feature].values
        y = df_train.loc[df_train["symbol"] == symbol, y_feature].values
        fr = df_train.loc[df_train["symbol"] == symbol, "funding_rate"].values
        cur_pnl = get_pnl_simple(x, fr, buythres, sellthres)
        total_pnl += cur_pnl
    pnl_rate = total_pnl / len(symbols)
    return pnl_rate * 12 / 5
df_scan = artool.analyze.scan.grid_scan(dims, obj_train)

# find optimal buythres and sellthres
df_tmp = df_scan.sort_values(by="score", ascending=False)
best_buy = df_tmp.iloc[0]["buythres"]
best_sell = df_tmp.iloc[0]["sellthres"]
max_agr = df_tmp.iloc[0]["score"]  # max annual growth rate
print(f"best_buy: {best_buy:.6f}, best_sell: {best_sell:.6f}")
print(f"max profit rate per year: {max_agr * 100:.4f} %")
opt_train = {
    "buythres": best_buy,
    "sellthres": best_sell,
    "max_agr": max_agr,
}

# Plot heat map
fig, ax = plt.subplots(figsize=(8, 8))
df_tmp = df_scan.round(6).pivot("buythres", "sellthres", "score")
sns.heatmap(df_tmp, ax=ax)
# add max_agr
ax.set_title(f"max_agr: {max_agr * 100:.4f} %")
fig.savefig(method_0_dir / "trend_sig.train.png")
plt.close(fig)

100%|██████████| 400/400 [00:23<00:00, 16.91it/s]


best_buy: 0.000211, best_sell: -0.000100
max profit rate per year: 3.2674 %


Scan test for validation

In [10]:
# scan buythres and sellthres
dims = {"buythres": buy_dim, "sellthres": sell_dim}
def obj_test(buythres, sellthres):
    total_pnl = 0
    for symbol in symbols:
        x = df_test.loc[df_test["symbol"] == symbol, x_feature].values
        y = df_test.loc[df_test["symbol"] == symbol, y_feature].values
        fr = df_test.loc[df_test["symbol"] == symbol, "funding_rate"].values
        cur_pnl = get_pnl_simple(x, fr, buythres, sellthres)
        total_pnl += cur_pnl
    pnl_rate = total_pnl / len(symbols)
    return pnl_rate * 4
df_scan = artool.analyze.scan.grid_scan(dims, obj_test)

# find optimal buythres and sellthres
df_tmp = df_scan.sort_values(by="score", ascending=False)
best_buy = df_tmp.iloc[0]["buythres"]
best_sell = df_tmp.iloc[0]["sellthres"]
max_agr = df_tmp.iloc[0]["score"]  # max annual growth rate
print(f"best_buy: {best_buy:.6f}, best_sell: {best_sell:.6f}")
print(f"max profit rate per year: {max_agr * 100:.4f} %")
opt_test = {
    "buythres": best_buy,
    "sellthres": best_sell,
    "max_agr": max_agr,
}

# Plot heat map
fig, ax = plt.subplots(figsize=(8, 8))
df_tmp = df_scan.round(6).pivot("buythres", "sellthres", "score")
sns.heatmap(df_tmp, ax=ax)
ax.set_title(f"max_agr: {max_agr * 100:.4f} %")
fig.savefig(method_0_dir / "trend_sig.test.png")
plt.close(fig)

100%|██████████| 400/400 [00:15<00:00, 25.45it/s]


best_buy: 0.000211, best_sell: -0.000047
max profit rate per year: 1.7009 %


Check optimal training result at test

In [11]:
agr_tr_opt = obj_test(opt_train["buythres"], opt_train["sellthres"])
agr_diff = (agr_tr_opt - opt_test["max_agr"]) / opt_test["max_agr"]
print(f"agr_diff: {agr_diff * 100:.4f} %")

agr_diff: -2.6782 %


## Method 0 - dynamic

In [4]:
method_0_dy_dir = Path("./method_0_dy")
method_0_dy_dir.mkdir(exist_ok=True)

toy

In [29]:
from artool.toy.toy_simu import get_pnl_1side_dynamic

buythres = 1e-4
sellthres = 0
total_pnl = 0

x_list = []
fr_list = []
for symbol in symbols:
    x = df_test.loc[df_test["symbol"] == symbol, x_feature].values
    fr = df_test.loc[df_test["symbol"] == symbol, "funding_rate"].values
    x_list.append(x.reshape(-1, 1))
    fr_list.append(fr.reshape(-1, 1))
x_2d = np.concatenate(x_list, axis=1)
fr_2d = np.concatenate(fr_list, axis=1)
print(f"x_2d.shape: {x_2d.shape}, fr_2d.shape: {fr_2d.shape}")
max_holds = 110
total_pnl = get_pnl_1side_dynamic(x_2d, fr_2d, buythres, sellthres, max_holds)
pnl_rate = total_pnl / max_holds
print(f"total_pnl: {total_pnl}, max_holds: {max_holds}")
print(f"pnl_rate (y): {pnl_rate * 4 * 100} %")

x_2d.shape: (276, 129), fr_2d.shape: (276, 129)
total_pnl: 0.4862682600000002, max_holds: 110
pnl_rate (y): 1.7682482181818189 %


Optimize on train

In [34]:
%load_ext autoreload
%autoreload 2
from artool.toy.toy_simu import get_pnl_1side_dynamic

# scan buythres and sellthres
dims = {"buythres": buy_dim, "sellthres": sell_dim}
max_holds = 50
def obj_train_dy(buythres, sellthres):
    x_list = []
    fr_list = []
    for symbol in symbols:
        x = df_train.loc[df_train["symbol"] == symbol, x_feature].values
        fr = df_train.loc[df_train["symbol"] == symbol, "funding_rate"].values
        x_list.append(x.reshape(-1, 1))
        fr_list.append(fr.reshape(-1, 1))
    x_2d = np.concatenate(x_list, axis=1)
    fr_2d = np.concatenate(fr_list, axis=1)
    total_pnl = get_pnl_1side_dynamic(x_2d, fr_2d, buythres, sellthres, max_holds)
    pnl_rate = total_pnl / max_holds
    return pnl_rate * 12 / 5
df_scan = artool.analyze.scan.grid_scan(dims, obj_train_dy)

# find optimal buythres and sellthres
df_tmp = df_scan.sort_values(by="score", ascending=False)
best_buy = df_tmp.iloc[0]["buythres"]
best_sell = df_tmp.iloc[0]["sellthres"]
max_agr = df_tmp.iloc[0]["score"]  # max annual growth rate
print(f"best_buy: {best_buy:.6f}, best_sell: {best_sell:.6f}")
print(f"max profit rate per year: {max_agr * 100:.4f} %")
opt_train = {
    "buythres": best_buy,
    "sellthres": best_sell,
    "max_agr": max_agr,
}

# Plot heat map
fig, ax = plt.subplots(figsize=(8, 8))
df_tmp = df_scan.round(6).pivot("buythres", "sellthres", "score")
sns.heatmap(df_tmp, ax=ax)
# add max_agr
ax.set_title(f"max_agr: {max_agr * 100:.4f} %")
fig.savefig(method_0_dy_dir / "trend_sig.train.png")
plt.close(fig)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 400/400 [00:15<00:00, 26.04it/s]


best_buy: 0.000368, best_sell: -0.000100
max profit rate per year: 3.9554 %


Optimize on test

In [36]:
# scan buythres and sellthres
dims = {"buythres": buy_dim, "sellthres": sell_dim}
max_holds = 50
def obj_test_dy(buythres, sellthres):
    x_list = []
    fr_list = []
    for symbol in symbols:
        x = df_test.loc[df_test["symbol"] == symbol, x_feature].values
        fr = df_test.loc[df_test["symbol"] == symbol, "funding_rate"].values
        x_list.append(x.reshape(-1, 1))
        fr_list.append(fr.reshape(-1, 1))
    x_2d = np.concatenate(x_list, axis=1)
    fr_2d = np.concatenate(fr_list, axis=1)
    total_pnl = get_pnl_1side_dynamic(x_2d, fr_2d, buythres, sellthres, max_holds)
    pnl_rate = total_pnl / max_holds
    return pnl_rate * 4
df_scan = artool.analyze.scan.grid_scan(dims, obj_test_dy)

# find optimal buythres and sellthres
df_tmp = df_scan.sort_values(by="score", ascending=False)
best_buy = df_tmp.iloc[0]["buythres"]
best_sell = df_tmp.iloc[0]["sellthres"]
max_agr = df_tmp.iloc[0]["score"]  # max annual growth rate
print(f"best_buy: {best_buy:.6f}, best_sell: {best_sell:.6f}")
print(f"max profit rate per year: {max_agr * 100:.4f} %")
opt_test = {
    "buythres": best_buy,
    "sellthres": best_sell,
    "max_agr": max_agr,
}

# Plot heat map
fig, ax = plt.subplots(figsize=(8, 8))
df_tmp = df_scan.round(6).pivot("buythres", "sellthres", "score")
sns.heatmap(df_tmp, ax=ax)
ax.set_title(f"max_agr: {max_agr * 100:.4f} %")
fig.savefig(method_0_dy_dir / "trend_sig.test.png")
plt.close(fig)

100%|██████████| 400/400 [00:10<00:00, 39.27it/s]


best_buy: 0.000263, best_sell: -0.000068
max profit rate per year: 2.1772 %


In [38]:
agr_tr_opt = obj_test_dy(opt_train["buythres"], opt_train["sellthres"])
agr_diff = (agr_tr_opt - opt_test["max_agr"]) / opt_test["max_agr"]
print(f"agr_diff: {agr_diff * 100:.4f} %")

agr_diff: -6.6598 %


## Method 1

Only use symbol with high correlation

In [9]:
method_1_dir = Path("./method_1")
method_1_dir.mkdir(exist_ok=True)

Check correlations

In [10]:
corr_dict = {}
for symbol in symbols:
    x = df_train.loc[df_train["symbol"] == symbol, x_feature].values
    y = df_train.loc[df_train["symbol"] == symbol, y_feature].values
    corr_dict[symbol] = np.corrcoef(x, y)[0, 1]
# Plot correlation_distribution
fig, ax = plt.subplots()
sns.kdeplot(list(corr_dict.values()), ax=ax)
ax.set_title("x vs y correlation")
fig.savefig(method_1_dir / "corr_dist.png")

Scan train and find optimal threshold

In [11]:
agr_dict_tr = {}
for corr_thre in [0.3, 0.5, 0.7, 0.72, 0.73, 0.74, 0.75, 0.76]:
    # corr_thre = 0.65
    symbols_high_corr = [k for k, v in corr_dict.items() if v > corr_thre]
    print(f"# num of symbols with corr > {corr_thre}: {len(symbols_high_corr)}")
    if len(symbols_high_corr) == 0:
        continue

    # scan buythres and sellthres
    dims = {"buythres": buy_dim, "sellthres": sell_dim}

    def obj_train(buythres, sellthres):
        total_pnl = 0
        for symbol in symbols_high_corr:
            x = df_train.loc[df_train["symbol"] == symbol, x_feature].values
            y = df_train.loc[df_train["symbol"] == symbol, y_feature].values
            fr = df_train.loc[df_train["symbol"] == symbol, "funding_rate"].values
            cur_pnl = get_pnl_simple(x, fr, buythres, sellthres)
            total_pnl += cur_pnl
        pnl_rate = total_pnl / len(symbols_high_corr)
        return pnl_rate * 12 / 5

    df_scan = artool.analyze.scan.grid_scan(dims, obj_train)

    # find optimal buythres and sellthres
    df_tmp = df_scan.sort_values(by="score", ascending=False)
    best_buy = df_tmp.iloc[0]["buythres"]
    best_sell = df_tmp.iloc[0]["sellthres"]
    max_agr = df_tmp.iloc[0]["score"]  # max annual growth rate
    print(f"best_buy: {best_buy:.6f}, best_sell: {best_sell:.6f}")
    print(f"max profit rate per year: {max_agr * 100:.4f} %")
    agr_dict_tr[corr_thre] = {
        "buythres": best_buy,
        "sellthres": best_sell,
        "max_agr": max_agr,
    }

    # Plot heat map
    fig, ax = plt.subplots(figsize=(8, 8))
    df_tmp = df_scan.round(6).pivot("buythres", "sellthres", "score")
    sns.heatmap(df_tmp, ax=ax)
    # add max_agr
    ax.set_title(f"max_agr: {max_agr * 100:.4f} %")
    fig.savefig(method_1_dir / f"trend_sig_corr_{corr_thre:.2f}.train.png")
    plt.close(fig)


# num of symbols with corr > 0.3: 117
best_buy: 0.000211, best_sell: -0.000100
max profit rate per year: 3.2202 %
# num of symbols with corr > 0.5: 79
best_buy: 0.000289, best_sell: -0.000100
max profit rate per year: 3.0630 %
# num of symbols with corr > 0.7: 11
best_buy: 0.000237, best_sell: -0.000100
max profit rate per year: 3.2992 %
# num of symbols with corr > 0.72: 9
best_buy: 0.000079, best_sell: -0.000100
max profit rate per year: 3.3402 %
# num of symbols with corr > 0.73: 7
best_buy: 0.000079, best_sell: -0.000100
max profit rate per year: 3.1702 %
# num of symbols with corr > 0.74: 5
best_buy: 0.000158, best_sell: -0.000100
max profit rate per year: 3.2488 %
# num of symbols with corr > 0.75: 3
best_buy: 0.000079, best_sell: -0.000058
max profit rate per year: 4.2238 %
# num of symbols with corr > 0.76: 1
best_buy: 0.000368, best_sell: 0.000100
max profit rate per year: 0.5050 %


In [12]:
# Plot max_agr vs corr_thre
fig, ax = plt.subplots()
sns.lineplot(list(agr_dict_tr.keys()), [v["max_agr"] for v in agr_dict_tr.values()], ax=ax)
ax.set_title("max_agr vs corr_thre")
fig.savefig(method_1_dir / "max_agr_vs_corr_thre.train.png")


/home/yangzhe/anaconda3/envs/aura/lib/python3.9/site-packages/seaborn/_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


Scan test

In [13]:
agr_dict_te = {}
for corr_thre in [0.3, 0.5, 0.7, 0.72, 0.73, 0.74, 0.75, 0.76]:
    # corr_thre = 0.65
    symbols_high_corr = [k for k, v in corr_dict.items() if v > corr_thre]
    print(f"# num of symbols with corr > {corr_thre}: {len(symbols_high_corr)}")
    if len(symbols_high_corr) == 0:
        continue

    # scan buythres and sellthres
    dims = {"buythres": buy_dim, "sellthres": sell_dim}

    def obj_test(buythres, sellthres):
        total_pnl = 0
        for symbol in symbols_high_corr:
            x = df_test.loc[df_test["symbol"] == symbol, x_feature].values
            y = df_test.loc[df_test["symbol"] == symbol, y_feature].values
            fr = df_test.loc[df_test["symbol"] == symbol, "funding_rate"].values
            cur_pnl = get_pnl_simple(x, fr, buythres, sellthres)
            total_pnl += cur_pnl
        pnl_rate = total_pnl / len(symbols_high_corr)
        return pnl_rate * 4

    df_scan = artool.analyze.scan.grid_scan(dims, obj_test)

    # find optimal buythres and sellthres
    df_tmp = df_scan.sort_values(by="score", ascending=False)
    best_buy = df_tmp.iloc[0]["buythres"]
    best_sell = df_tmp.iloc[0]["sellthres"]
    max_agr = df_tmp.iloc[0]["score"]  # max annual growth rate
    print(f"best_buy: {best_buy:.6f}, best_sell: {best_sell:.6f}")
    print(f"max profit rate per year: {max_agr * 100:.4f} %")
    agr_dict_te[corr_thre] = {
        "buythres": best_buy,
        "sellthres": best_sell,
        "max_agr": max_agr,
    }

    # Plot heat map
    fig, ax = plt.subplots(figsize=(8, 8))
    df_tmp = df_scan.round(6).pivot("buythres", "sellthres", "score")
    sns.heatmap(df_tmp, ax=ax)
    # add max_agr
    ax.set_title(f"max_agr: {max_agr * 100:.4f} %")
    fig.savefig(method_1_dir / f"trend_sig_corr_{corr_thre:.2f}.test.png")
    plt.close(fig)


# num of symbols with corr > 0.3: 117
best_buy: 0.000211, best_sell: -0.000047
max profit rate per year: 1.6829 %
# num of symbols with corr > 0.5: 79
best_buy: 0.000237, best_sell: -0.000047
max profit rate per year: 1.5809 %
# num of symbols with corr > 0.7: 11
best_buy: 0.000132, best_sell: -0.000100
max profit rate per year: 2.2202 %
# num of symbols with corr > 0.72: 9
best_buy: 0.000132, best_sell: -0.000100
max profit rate per year: 2.5534 %
# num of symbols with corr > 0.73: 7
best_buy: 0.000132, best_sell: -0.000100
max profit rate per year: 2.4144 %
# num of symbols with corr > 0.74: 5
best_buy: 0.000132, best_sell: -0.000026
max profit rate per year: 1.8816 %
# num of symbols with corr > 0.75: 3
best_buy: 0.000158, best_sell: -0.000026
max profit rate per year: 3.4199 %
# num of symbols with corr > 0.76: 1
best_buy: 0.000132, best_sell: -0.000005
max profit rate per year: 3.1941 %


In [14]:
# Plot max_agr vs corr_thre
fig, ax = plt.subplots()
ax.plot(list(agr_dict_tr.keys()), [v["max_agr"] for v in agr_dict_tr.values()], label="train")
ax.plot(list(agr_dict_te.keys()), [v["max_agr"] for v in agr_dict_te.values()], label="test")
ax.legend()
ax.set_title("max_agr vs corr_thre")
fig.savefig(method_1_dir / "max_agr_vs_corr_thre.png")


In [15]:
df_test.head()

,symbol,funding_timestamp,funding_rate,index_price,mark_price,funding_rate_expcumsum_0.5,funding_rate_expcumsum_1,funding_rate_expcumsum_2,funding_rate_expcumsum_3,funding_rate_expcumsum_5,funding_rate_expcumsum_10,funding_rate_expcumsum_20,funding_rate_expcumsum_50,funding_rate_rol_mean_3,funding_rate_rol_std_3,funding_rate_rol_max_3,funding_rate_rol_min_3,funding_rate_rol_skew_3,funding_rate_rol_mean_5,funding_rate_rol_std_5,funding_rate_rol_max_5,funding_rate_rol_min_5,funding_rate_rol_skew_5,funding_rate_rol_kurt_5,funding_rate_rol_mean_10,funding_rate_rol_std_10,funding_rate_rol_max_10,funding_rate_rol_min_10,funding_rate_rol_skew_10,funding_rate_rol_kurt_10,funding_rate_rol_mean_20,funding_rate_rol_std_20,funding_rate_rol_max_20,funding_rate_rol_min_20,funding_rate_rol_skew_20,funding_rate_rol_kurt_20,funding_rate_rol_mean_50,funding_rate_rol_std_50,funding_rate_rol_max_50,funding_rate_rol_min_50,funding_rate_rol_skew_50,funding_rate_rol_kurt_50,funding_rate_future_1,funding_rate_future_3,funding_rate_future_5,funding_rate_future_10,index_price_expcumsum_0.5,index_price_expcumsum_1,index_price_expcumsum_2,index_price_expcumsum_3,index_price_expcumsum_5,index_price_expcumsum_10,index_price_expcumsum_20,index_price_expcumsum_50,index_price_rol_mean_3,index_price_rol_std_3,index_price_rol_max_3,index_price_rol_min_3,index_price_rol_skew_3,index_price_rol_mean_5,index_price_rol_std_5,index_price_rol_max_5,index_price_rol_min_5,index_price_rol_skew_5,index_price_rol_kurt_5,index_price_rol_mean_10,index_price_rol_std_10,index_price_rol_max_10,index_price_rol_min_10,index_price_rol_skew_10,index_price_rol_kurt_10,index_price_rol_mean_20,index_price_rol_std_20,index_price_rol_max_20,index_price_rol_min_20,index_price_rol_skew_20,index_price_rol_kurt_20,index_price_rol_mean_50,index_price_rol_std_50,index_price_rol_max_50,index_price_rol_min_50,index_price_rol_skew_50,index_price_rol_kurt_50,mark_price_expcumsum_0.5,mark_price_expcumsum_1,mark_price_expcumsum_2,mark_price_expcumsum_3,mark_price_expcumsum_5,mark_price_expcumsum_10,mark_price_expcumsum_20,mark_price_expcumsum_50,mark_price_rol_mean_3,mark_price_rol_std_3,mark_price_rol_max_3,mark_price_rol_min_3,mark_price_rol_skew_3,mark_price_rol_mean_5,mark_price_rol_std_5,mark_price_rol_max_5,mark_price_rol_min_5,mark_price_rol_skew_5,mark_price_rol_kurt_5,mark_price_rol_mean_10,mark_price_rol_std_10,mark_price_rol_max_10,mark_price_rol_min_10,mark_price_rol_skew_10,mark_price_rol_kurt_10,mark_price_rol_mean_20,mark_price_rol_std_20,mark_price_rol_max_20,mark_price_rol_min_20,mark_price_rol_skew_20,mark_price_rol_kurt_20,mark_price_rol_mean_50,mark_price_rol_std_50,mark_price_rol_max_50,mark_price_rol_min_50,mark_price_rol_skew_50,mark_price_rol_kurt_50
446,1INCHUSDT,1.654013e+15,-0.000285,0.985156,0.984500,-0.000045,-0.000168,-0.000432,-0.000697,-0.001255,-0.002754,-0.005885,-0.013380,-0.000306,0.000027,-0.000285,-0.000337,-1.482026,-0.000282,0.000059,-0.000182,-0.000337,1.625728,3.130746,-0.000263,0.000105,-0.000046,-0.000437,0.610480,1.468947,-0.000320,0.000105,-0.000046,-0.000473,0.737071,1.047810,-0.000292,0.000135,4.504000e-05,-0.000587,0.140142,0.196976,-0.000212,-0.001721,-0.003987,-0.006779,0.154529,0.574545,1.506958,2.445844,4.316144,9.017988,18.724337,52.879360,0.988579,0.015243,1.005242,0.975339,0.959507,0.971105,0.031312,1.005242,0.920738,-1.155721,2.185708,0.941277,0.037901,1.005242,0.904884,0.652992,-1.471701,0.940164,0.034692,1.005242,0.898015,0.520266,-1.305268,0.944872,0.034275,1.023742,0.898015,0.657182,-0.689396,0.154420,0.574103,1.505686,2.443684,4.312158,9.009306,18.705565,52.828010,0.987733,0.015210,1.004300,0.9744,0.913380,0.970219,0.031383,1.004300,0.919696,-1.164749,2.188844,0.940362,0.037949,1.0043,0.903782,0.652808,-1.478507,0.939268,0.034731,1.0043,0.8974,0.522108,-1.304442,0.943878,0.034300,1.022893,0.8974,0.656156,-0.697208
447,1INCHUSDT,1.654042e+15,-0.000212,0.996898,0.995816,-0.000035,-0.000140,-0.000390,-0.000651,-0.001201,-0.002683,-0.005799,

In [16]:
df_tmp = pd.read_csv("/home/shared/coin/tardis_derivative_ticker/2022-09-30/BTCUSDT.csv")

In [17]:
df_tmp.sample(10)

,exchange,symbol,timestamp,local_timestamp,funding_timestamp,funding_rate,predicted_funding_rate,open_interest,last_price,index_price,mark_price
107041,binance-futures,BTCUSDT,1664545501000000,1664545501097256,1.664554e+15,0.000100,NaN,137593.008,19336.8,19344.434803,19337.473973
86486,binance-futures,BTCUSDT,1664536855001000,1664536855093324,1.664554e+15,0.000100,NaN,136802.930,19463.8,19469.221128,19463.900000
180430,binance-futures,BTCUSDT,1664575508069000,1664575508104436,1.664582e+15,0.000049,NaN,127626.320,19310.6,19323.028430,19312.988562
67187,binance-futures,BTCUSDT,1664527740333000,1664527740339675,1.664554e+15,0.000100,NaN,136193.505,19554.0,19558.001146,19554.600000
116652,binance-futures,BTCUSDT,1664549175002000,1664549175087925,1.664554e+15,0.000100,NaN,140702.694,19839.4,19845.965141,19839.900000
73421,binance-futures,BTCUSDT,1664530679006000,1664530679133637,1.664554e+15,0.000100,NaN,137170.498,19523.6,19532.460599,19523.600000
71950,binance-futures,BTCUSDT,1664529942340000,1664529942526704,1.664554e+15,0.000100,NaN,137196.183,19542.3,19553.341997,19545.092787
23012,binance-futures,BTCUSDT,1664506305452000,1664506305671239,1.664525e+15,0.000100,NaN,131472.241,19393.5,19398.222597,19393.500536
119827,binance-futures,BTCUSDT,1664550309010000,1664550309078521,1.664554e+15,0.000100,NaN,143988.177,20029.5,20041.411730,20030.616980
126057,binance-futures,BTCUSDT,1664552621879000,1664552621997027,1.664554e+15,0.000100,NaN,151324.788,19887.3,19889.023707,19886.800000


In [18]:
df_tmp.describe()

,timestamp,local_timestamp,funding_timestamp,funding_rate,predicted_funding_rate,open_interest,last_price,index_price,mark_price
count,1.947710e+05,1.947710e+05,1.947680e+05,194768.000000,0.0,194768.000000,194770.000000,194768.000000,194768.000000
mean,1.664540e+15,1.664540e+15,1.664554e+15,0.000091,NaN,135590.626439,19524.357155,19531.993754,19525.386661
std,2.458387e+10,2.458386e+10,2.325570e+10,0.000017,NaN,5676.046384,176.173889,176.465854,176.325656
min,1.664496e+15,1.664496e+15,1.664496e+15,0.000033,NaN,125456.884000,19147.400000,19172.642065,19165.674014
25%,1.664519e+15,1.664519e+15,1.664525e+15,0.000087,NaN,131284.937000,19400.600000,19407.955598,19401.500000
50%,1.664542e+15,1.664542e+15,1.664554e+15,0.000100,NaN,135882.031000,19480.200000,19486.798042,19480.800000
75%,1.664561e+15,1.664561e+15,1.664582e+15,0.000100,NaN,138231.992000,19609.100000,19617.592682,19611.020420
max,1.664582e+15,1.664582e+15,1.664582e+15,0.000100,NaN,151564.560000,20166.100000,20171.151555,20167.000000
